# A5

In [1]:
# Standard imports
import numpy as np
import copy
import time
import matplotlib.pyplot as plt
%matplotlib inline

# Q1

## Read in the data

The code below creates two lists:
  - `sentences`, and
  - `next_chars`
  
Each list element represents a sequences of characters. There are 3 ways to represent a character:
1. As a string, eg. `'b'`
2. As an index to a character set, eg. `2`
3. As a one-hot vector, eg. `[0, 0, 1, 0, ...]`

The lists `sentences` and `next_chars` store the characters as indices. The utility functions
  - `char2vec`
  - `index2vec`
  - `vec2char`
  - `vec2index`
  
transform the characters between the 3 representations. You can also use the dictionaries `char_indices` and `indices_char` to convert between a string character and and index. The code below contains some examples.

In [2]:
import re
text = open('origin_of_species.txt').read().lower()
chars = sorted(list(set(text)))
chars.insert(0, "\0") #Add newline character
vocab_size = len(chars)

char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
idx = [char_indices[c] for c in text]

# Let's simplify it by keeping only letters and spaces
filt_idx = []
for i in idx:
    if i<=24:
        filt_idx.append(2)
    elif i>24:
        filt_idx.append(i)
blah = ''.join([indices_char[f] for f in filt_idx])
text = re.sub(' +', ' ', blah)
chars = sorted(list(set(text)))
vocab_size = len(chars)
print('Character set: '+''.join(chars)+' (first char is a space)')

char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
idx = [char_indices[c] for c in text]

print('There are '+str(vocab_size)+' characters in our character set')

''.join(indices_char[i] for i in idx[:70])

def char2vec(c):
    v = np.zeros(vocab_size)
    v[char_indices[c]] = 1.
    return v

def index2vec(i):
    v = np.zeros(vocab_size)
    v[i] = 1.
    return v

def vec2index(v):
    i = np.argmax(v)
    return i

def vec2char(v):
    return indices_char[vec2index(v)]

'''Form the dataset in sentences'''
sentences_length = 10
sentences = []
next_chars = []
for i in range(0, 5000 - sentences_length + 1):
    sentences.append(idx[i: i + sentences_length]) #Assume a sentence is made of X characters
    next_chars.append(idx[i + 1: i + sentences_length + 1]) #Offset by 1 to the right for the target

sentences = np.concatenate([[np.array(o)] for o in sentences[:-2]])
next_chars = np.concatenate([[np.array(o)] for o in next_chars[:-2]])
sentences.shape, next_chars.shape

def read_sentence(idx):
    return ''.join(indices_char[i] for i in idx)

print('Here is how you can view one of the samples:')
print('Sample input: ['+read_sentence(sentences[0])+']')

print(read_sentence(sentences[2]))
print(read_sentence(next_chars[2]))

Character set:  abcdefghijklmnopqrstuvwxyz (first char is a space)
There are 27 characters in our character set
Here is how you can view one of the samples:
Sample input: [on the ori]
 the origi
the origin


## Some utility functions

In [3]:
def sigma(z):
    return np.clip(z, a_min=0, a_max=None)  # ReLU
    #return 1./(1+np.exp(-z))  # use this for logistic

def sigma_primed(y):
    return np.clip(np.sign(y), a_min=0, a_max=1)  # Derivative of ReLU
    #return y*(1.-y)  # use this for logistic

def softmax(z):
    ez = np.exp(z)
    denom = np.sum(ez)
    return ez / denom

def CrossEntropy(y, t):
    return - sum(t * np.log(y))

## (a) Complete BPTT

In [4]:
class RNN():
    
    def __init__(self, dims, seq_length=10):
        '''
         Input:
           dims is [X, H, Y], where the input has layer has X neurons, the
                hidden layer has H neurons, and the output layer has Y neurons.
           seq_length is how many steps to unroll the RNN through time
                (this is the same as tau in the lecture notes)
        '''
        self.X, self.H, self.Y = dims
        self.seq_length = seq_length
        # Input layer
        self.xs = [np.zeros(self.X) for x in range(seq_length)] # activity
        # Hidden layer
        self.hs = [np.zeros(self.H) for x in range(seq_length)] # activity
        # Output layer
        self.ys = [np.zeros(self.Y) for x in range(seq_length)] # activity
        
        # Connection weights
        self.U = np.random.normal(size=[self.H, self.X])/np.sqrt(self.X) # input->hidden
        self.W = np.random.normal(size=[self.H, self.H])/np.sqrt(self.H) # hidden->hidden
        self.V = np.random.normal(size=[self.Y, self.H])/np.sqrt(self.H) # hidden->output
        self.b = np.zeros(self.H) # biases for hidden nodes
        self.c = np.zeros(self.Y) # biases for output nodes
    
    
    def ForwardTT(self, seq_in):
        '''
         i = ForwardTT(seq_in)
        
         Propagates the RNN forward through time, saving all the intermediate
         states that will be needed for backprop through time (BPTT).
        
         Input:
           seq_in is a vector of indecies, with self.seq_length elements.
        
         Output:
           i is the index of the character predicted to follow the input.
         
         This method's main purpose is to update the states of the activites
         in the time-unrolled network.
        '''
        self.xs[0] = index2vec(seq_in[0]) # convert to character vector
        
        # Starting input current for hidden nodes
        ss = np.dot(self.U, self.xs[0]) + self.b
        self.hs[0] = sigma(ss)  # Activation of hidden nodes
        
        # Input current for output nodes
        zs = np.dot(self.V, self.hs[0]) + self.c
        self.ys[0] = softmax(zs)  # Activation of output nodes
        
        # Now process forward in time
        for i in range(1, self.seq_length):
            self.xs[i] = index2vec(seq_in[i])  # input vector
            
            # Input current for hidden nodes, including recurrent connections
            ss = np.dot(self.U, self.xs[i]) + np.dot(self.W, self.hs[i-1]) + self.b
            self.hs[i] = sigma(ss)  # Activation
            
            # Input current for output nodes
            zs = np.dot(self.V, self.hs[i]) + self.c
            self.ys[i] = softmax(zs)  # Activation
            
        # Might as well output the final state of the output
        return vec2index(self.ys[-1])
    
    
    def Generate(self, n=1):
        '''
         c = Generate(n=1)
         
         Runs the RNN from the last state after running ForwardTT, outputting
         the next n characters.
         
         Input:
           n is the number of characters you want to predict
           
         Output:
           c is a string of n characters
        '''
        y = self.ys[-1]  # Final output of ForwardTT
        c = vec2char(y)  # Convert it to a character string
        h = self.hs[-1]  # Starting with last hidden state...
        # Loop forward in time
        # (no need to record states, since we won't be doing BPTT)
        for nn in range(n-1):
            x = copy.copy(y)  # Use last output as next input
            
            # Input current for hidden nodes
            s = np.dot(self.U, x) + np.dot(self.W, h) + self.b
            h = sigma(s)  # Activation
            
            # Input current for output nodes
            z = np.dot(self.V, h) + self.c
            y = softmax(z)  # Activation
            
            # And add the next character to our output string
            c += vec2char(y)
            
        return c
    
    
    def BPTT(self, seq_in, seq_out):
        '''
         BPTT(seq_in, seq_out)
         
         Performs backprop through time on one sample given by the input and
         output sequence.
         
         Input:
           seq_in is a vector of indices specifying the input sequence of
                   characters.
           seq_out is a vector of indices specifying the output sequence of
                   characters. Typically, seq_out is the same as seq_in, but
                   shifted by 1 character.
         
         Output:
           None, but the connection weights and biases are updated.
        '''
        # Initialize gradients to zero
        self.dEdV = np.zeros(np.shape(self.V))
        self.dEdW = np.zeros(np.shape(self.W))
        self.dEdU = np.zeros(np.shape(self.U))
        self.dEdb = np.zeros(np.shape(self.b))
        self.dEdc = np.zeros(np.shape(self.c))
        
    
        # ===================
        # ===================
        # =  YOUR CODE HERE =
        # ===================
        # ===================
        
        dEdz = [self.ys[i] - index2vec(seq_out[i]) for i in range(self.seq_length)]
        dEds = [0 for i in range(self.seq_length)]
        s = [0 for i in range(self.seq_length)]
        
        # compute s
        s[0] = np.dot(self.U, self.xs[0]) + self.b
        for i in range(1, self.seq_length):
            s[i] = np.dot(self.W, self.hs[i - 1]) + np.dot(self.U, self.xs[i]) + self.b
        
        # compute dEds
        dEds[-1] = sigma_primed(s[-1]) * np.dot(self.V.T, dEdz[-1])
        for i in range(self.seq_length - 2, -1, -1):
            dEds[i] = sigma_primed(s[i]) * (np.dot(self.V.T, dEdz[i]) + np.dot(self.W.T, dEds[i + 1]))
        
        # compute dEdV
        for i in range(self.seq_length):
            self.dEdV += np.outer(dEdz[i], self.hs[i])
        
        # compute dEdU
        for i in range(self.seq_length):
            self.dEdU += np.outer(dEds[i], self.xs[i])
        
        # compute dEdW
        for i in range(self.seq_length - 1):
            self.dEdW += np.outer(dEds[i + 1], self.hs[i])
        
        # compute dEdb
        for i in range(self.seq_length):
            self.dEdb += dEds[i]

        # compute dEdc
        for i in range(self.seq_length):
            self.dEdc += dEdz[i]    
        
    def Evaluate(self, train_in, train_out):
        '''
         loss = Evaluate(train_in, train_out)
         
         Evaluates the network on the supplied dataset.
         
         Input:
           train_in is a list of input sequences (see ForwardTT for format of input)
           train_out is the corresponding list of output sequences
           
         Output:
           loss is the average cross entropy
        '''
        val = 0.
        for x, t in zip(train_in, train_out):
            self.ForwardTT(x)
            for i in range(self.seq_length):
                val += CrossEntropy(self.ys[i], index2vec(t[i]))
        return val/len(train_in)
    
    
    def Train(self, train_in, train_out, kappa=0.05, epochs=1):
        '''
         Train(train_in, train_out, kappa=0.05, epochs=1)
         
         Performs epochs of gradient descent, performing BPTT after each sample.
         
         Input:
           train_in and train_out is the training dataset
           kappa is the learning rate
           epochs is the number of times to go through the dataset
           
         Output:
           None, but the connection weights and biases are updated
        '''
        # Loop over epochs
        for e in range(epochs):
            
            # Shuffle the training data
            data_shuffled = list(zip(train_in, train_out))
            np.random.shuffle(data_shuffled)

            for x, t in data_shuffled:
#                 print(read_sentence(x))
#                 print(read_sentence(t))
#                 time.sleep(10)
                
                self.ForwardTT(x)  # Forward through time
                self.BPTT(x, t)    # Backprop through time
                # Note that BPTT starts by resetting the gradients to zero.

                # Apply update to connection weights and biases
                self.V -= kappa*self.dEdV
                self.U -= kappa*self.dEdU
                self.W -= kappa*self.dEdW
                self.b -= kappa*self.dEdb
                self.c -= kappa*self.dEdc

            print('Epoch '+str(e)+', Loss = '+str(self.Evaluate(train_in, train_out)))

## (b) Create the RNN

In [5]:
# YOUR CODE HERE
net = RNN(dims = [27, 400, 27])

## (c) Train

In [6]:
# YOUR CODE HERE
net.Train(sentences, next_chars, kappa = 0.001, epochs = 15)

Epoch 0, Loss = 20.663121898591335
Epoch 1, Loss = 17.74642826056183
Epoch 2, Loss = 15.498805268678334
Epoch 3, Loss = 14.005227518206434
Epoch 4, Loss = 12.373958499458029
Epoch 5, Loss = 11.268339718996147
Epoch 6, Loss = 10.585846943425175
Epoch 7, Loss = 10.035979235664657
Epoch 8, Loss = 9.152795052629132
Epoch 9, Loss = 8.8711732809374
Epoch 10, Loss = 8.574602669410861
Epoch 11, Loss = 8.315856427370166
Epoch 12, Loss = 8.139035755968058
Epoch 13, Loss = 7.923261172784949
Epoch 14, Loss = 7.840028566598258


In [215]:
net.Train(sentences, next_chars, kappa = 0.001, epochs = 1)

Epoch 0, Loss = 17.930315917510516


In [170]:
# You might opt to have more than one train command, using different
# learning rates and numbers of epochs. Each one builds on the results
# from the last run.
net.Train(sentences, next_chars, kappa = 0.0005, epochs = 1)

Epoch 0, Loss = 23.184445122949022


## (d) Evaluate

In [18]:
# A sample continuation.
n = 38
b = net
b.ForwardTT(sentences[n])
blah = read_sentence(sentences[n])

print('Input:      ' + blah + '$')
print('Prediction: ' + blah + b.Generate(5) + '$')
print('Actual:     ' + blah + read_sentence(sentences[n+10]) + '$')

Input:      n introduc$
Prediction: n introduction $
Actual:     n introduction when $


In [19]:
# Another example.
blah = 'harles dar'
x = [char_indices[c] for c in blah]
b.ForwardTT(x)
print(blah + '$')
print(blah + b.Generate(10) + '$')

harles dar$
harles darwin iowa s$


In [20]:
# YOUR CODE HERE
cnt = 0
for i in range(len(sentences)):
    net.ForwardTT(sentences[i])
    prediction = net.Generate()
    nextchar = vec2char(index2vec(next_chars[i][-1]))
    if prediction == nextchar:
        cnt += 1

print(cnt)
print(cnt / len(sentences))

4827
0.9675285628382442


# Q2

In [34]:
# You may include some Python code to help you.

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def tanh(z):
    return np.tanh(z)

def ComputeGateOutput(xt, ct_1, ht_1):
    
    Wf = np.array([0, 8, 0, 0])
    Wi = np.array([0, 0, 9, 0])
    Wo = np.array([0, 0, 0, 10])
    Wc = np.array([1, 0, 0, 0])

    bf = np.array([-4])
    bi = np.array([-4.5])
    bo = np.array([-5])
    bc = np.array([0])

    vt = np.concatenate((ht_1, xt))

    ft = sigmoid(np.dot(Wf, vt) + bf)
    it = sigmoid(np.dot(Wi, vt) + bi)
    ot = sigmoid(np.dot(Wo, vt) + bo)

    cqt = tanh(np.dot(Wc, vt) + bc)
    ct = ft * ct_1 + it * cqt
    ht = ot * tanh(ct)
    
    return ft, it, ot, ct, ht

h0 = np.array([0.05])
c0 = np.array([-0.02])

## (a)

### (i)

In [44]:
# You can include some code, if you want.

xt = np.array([1, 0, 0])

ft, it, ot, ct, ht = ComputeGateOutput(xt, c0, h0)

print('ft = ' + str(ft) + ', it = ' + str(it) + ', ot = ' + str(ot))
print('Ct = ' + str(ct) + ', ht = ' + str(ht))
print('Ct-1 = ' + str(c0) + ', ht-1 = ' + str(h0) + ', xt = ' + str(xt))

xt = np.array([0, 1, 0])

ft, it, ot, ct, ht = ComputeGateOutput(xt, c0, h0)

print('---------------------------------------------------------')
print('ft = ' + str(ft) + ', it = ' + str(it) + ', ot = ' + str(ot))
print('Ct = ' + str(ct) + ', ht = ' + str(ht))
print('Ct-1 = ' + str(c0) + ', ht-1 = ' + str(h0) + ', xt = ' + str(xt))

xt = np.array([0, 0, 1])

ft, it, ot, ct, ht = ComputeGateOutput(xt, c0, h0)

print('---------------------------------------------------------')
print('ft = ' + str(ft) + ', it = ' + str(it) + ', ot = ' + str(ot))
print('Ct = ' + str(ct) + ', ht = ' + str(ht))
print('Ct-1 = ' + str(c0) + ', ht-1 = ' + str(h0) + ', xt = ' + str(xt))

ft = [0.98201379], it = [0.01098694], ot = [0.00669285]
Ct = [-0.01909139], ht = [-0.00012776]
Ct-1 = [-0.02], ht-1 = [0.05], xt = [1 0 0]
---------------------------------------------------------
ft = [0.01798621], it = [0.98901306], ot = [0.00669285]
Ct = [0.04904976], ht = [0.00032802]
Ct-1 = [-0.02], ht-1 = [0.05], xt = [0 1 0]
---------------------------------------------------------
ft = [0.01798621], it = [0.01098694], ot = [0.99330715]
Ct = [0.00018917], ht = [0.0001879]
Ct-1 = [-0.02], ht-1 = [0.05], xt = [0 0 1]


YOUR ANSWER HERE.

### (ii)

YOUR ANSWER HERE.

### (iii)

YOUR ANSWER HERE.

## (b)

YOUR ANSWER HERE.

## (c)

YOUR ANSWER HERE